---
# Explicación del Script `app.py` (Aplicación Streamlit Simplificada)

Este script (`app.py`) es el código principal de la **aplicación interactiva Streamlit** de tu proyecto. Su función es proporcionar una interfaz de usuario donde las personas pueden ingresar el peso de un cangrejo y obtener predicciones visuales y numéricas sobre el peso de sus partes (carne, vísceras, caparazón), utilizando el modelo previamente entrenado.

---

## 1. Propósito General de la Aplicación

El objetivo principal de esta aplicación Streamlit es:
* **Proporcionar una interfaz amigable** para que los usuarios interactúen con el modelo de regresión de cangrejos.
* **Cargar los recursos de Machine Learning** (lambdas de Box-Cox y modelos de regresión) que fueron pre-entrenados por el script `preprocess_and_save_simplified_model.py`.
* **Validar la entrada del usuario** para asegurar que esté dentro de un rango aceptable.
* **Realizar predicciones** del peso de las partes del cangrejo basadas en el peso total ingresado por el usuario.
* **Detransformar** las predicciones a la escala original (gramos) para una fácil interpretación.
* **Ajustar las predicciones** para que su suma sea igual al peso total ingresado.
* **Visualizar los resultados** mediante gráficos de barras y circulares.
* **Mostrar un resumen** de la "producción" (carne, material de aprovechamiento, desperdicio).
* **Presentar las métricas de evaluación** del modelo para dar contexto sobre su rendimiento.

---

## 2. Configuración Inicial y Carga de Recursos

### A. Configuración de Rutas
El script comienza definiendo las rutas y nombres de los archivos (`lambdas`, `modelos`, `métricas`) que se generaron y guardaron en el script de preprocesamiento. Es crucial que estos nombres y rutas coincidan.

### B. Configuración de Streamlit
`st.set_page_config(layout="wide", page_title="Análisis de Cangrejos Simplificado")` configura el diseño de la página para que ocupe todo el ancho disponible, lo cual es ideal para mostrar múltiples elementos como gráficos y tablas.

### C. Carga de Recursos ML (`@st.cache_resource`)
* La función `load_ml_resources()` es la encargada de cargar los archivos `.joblib` que contienen los lambdas de Box-Cox, los modelos de regresión lineal entrenados y las métricas de evaluación.
* **`@st.cache_resource`:** Este decorador es muy importante en Streamlit. Le indica a Streamlit que solo cargue estos recursos pesados (modelos, lambdas) **una vez** al inicio de la aplicación, incluso si el script se vuelve a ejecutar debido a interacciones del usuario. Esto mejora drásticamente el rendimiento de la aplicación.
* **Manejo de Errores:** Incluye bloques `try-except` para detectar si los archivos no se encuentran, informando al usuario y deteniendo la ejecución de la app si los recursos esenciales no están disponibles (p. ej., si `preprocess_and_save_simplified_model.py` no se ha ejecutado).

---

## 3. Lógica de Predicción y Transformación

### A. Definición de Features y Targets
Se especifican los nombres de la única feature (`new_weight_boxcox`) y de las targets transformadas (`Shucked Weight_boxcox`, etc.), así como sus nombres originales.

### B. `prepare_user_input_for_model`
* Esta función toma el `input_weight` (el peso ingresado por el usuario) y lo transforma utilizando el **lambda de Box-Cox** correspondiente para `new_weight`.
* Es vital que esta transformación se haga exactamente igual que en el script de preprocesamiento, incluyendo cualquier `offset` para valores no positivos, aunque aquí el rango de entrada minimiza la necesidad de un offset.
* El resultado es un `DataFrame` con la entrada del usuario en la escala transformada, lista para ser utilizada por el modelo.

### C. Realización de Predicciones
* Una vez que la entrada del usuario está preparada, se itera sobre cada uno de los modelos entrenados (para carne, vísceras, caparazón).
* Cada modelo predice el peso de su respectiva parte en la **escala Box-Cox transformada**.

### D. Transformación Inversa (`inv_boxcox`) y Post-procesamiento
* Las predicciones obtenidas de los modelos están en la escala transformada y no son directamente interpretables en gramos.
* Se utiliza `scipy.special.inv_boxcox(transformed_val, lmbda)` para **detransformar** estas predicciones de nuevo a la escala original en gramos, utilizando los `lambdas` previamente cargados.
* **Asegurar No-Negatividad:** Se aplica `max(0, detransformed_val)` para garantizar que ningún peso predicho sea negativo, lo cual no tendría sentido físico.
* **Ajuste de Suma de Pesos:** Un paso crucial de post-procesamiento es el **ajuste de las predicciones**. La suma de los pesos predichos de las partes de un cangrejo debe ser igual al peso total del cangrejo ingresado por el usuario. Dado que los modelos predicen las partes de forma independiente, sus sumas rara vez coincidirán perfectamente con el peso total original. Este bloque recalcula un `adjustment_factor` y lo aplica a todas las predicciones para que su suma sea exactamente igual al `new_weight_input_for_prediction`. Esto hace que las predicciones sean coherentes con la entrada del usuario.

---

## 4. Interfaz de Usuario y Visualización

### A. Sidebar para Entrada de Usuario
* La barra lateral (`st.sidebar`) contiene el campo de entrada para el `New Weight` del cangrejo.
* Se establecen límites fijos para la entrada (`MIN_NEW_WEIGHT_APP = 5.0` y `MAX_NEW_WEIGHT_APP = 85.0`). Estos límites coinciden con el rango de pesos para los cuales el modelo fue **entrenado**. Esto es vital para evitar que el modelo haga **extrapolaciones** poco fiables fuera de su rango de aprendizaje.
* Un botón "Calcular Predicciones" inicia el proceso.
* **`st.session_state`:** Se utiliza para manejar el estado de la aplicación, como `run_prediction`, lo que permite que las predicciones solo se muestren cuando el botón ha sido presionado y los valores son válidos.

### B. Validación de Entrada
Antes de realizar cualquier predicción, el script valida que el `new_weight_input` esté dentro del rango `[5.0, 85.0]`. Si está fuera de este rango, muestra un mensaje de error y no procede con la predicción, reforzando el uso del modelo dentro de sus límites de entrenamiento.

### C. Visualización de Resultados
* Se utilizan las funciones `plot_bar_chart` y `generate_pie_chart` (que no cambian mucho de versiones anteriores) para mostrar visualmente la comparación y distribución de los pesos predichos.
* Los gráficos se muestran en dos columnas (`st.columns`) para un diseño organizado.

### D. Resumen de Producción
Se calcula y muestra una tabla con un "Resumen de la Producción", categorizando los pesos predichos en:
* **Carne producida:** `Shucked Weight`
* **Material de aprovechamiento:** `Shucked Weight` + `Shell Weight` (asumiendo que la cáscara también podría tener un uso)
* **Desperdicio:** `Viscera Weight`

### E. Métricas del Modelo
Finalmente, si las métricas fueron cargadas exitosamente, se muestra una tabla resumen con el MAE y R² para los conjuntos de entrenamiento y prueba para cada variable objetivo. Esto da al usuario o desarrollador una idea de la precisión del modelo.

---

## 7. ¿Cómo Influyó este Script en el Proyecto?

Este `app.py` transforma un análisis de datos y un modelo de Machine Learning en una **herramienta interactiva y útil**:

* **Accesibilidad:** Permite que cualquier persona, sin necesidad de conocimientos de programación, use el modelo para obtener predicciones.
* **Decisiones Informadas:** Al proporcionar predicciones claras y visualizaciones, ayuda a la toma de decisiones, por ejemplo, en la estimación de la cantidad de carne obtenible de un cangrejo de cierto peso.
* **Validación de Datos:** El manejo de límites de entrada y mensajes de error guía al usuario a usar el modelo dentro de sus capacidades, mejorando la confianza en los resultados.
* **Demostración del Valor:** Es la "cara" del proyecto, demostrando cómo los modelos de ML pueden aplicarse para resolver problemas prácticos (en este caso, la estimación de componentes del cangrejo).
* **Modularidad:** Al cargar recursos pre-entrenados, mantiene la aplicación ágil y separada del proceso de entrenamiento, lo cual es una buena práctica en el desarrollo de software.

---
## Explicación del Fragmento de Código: Factor de Ajuste en Predicciones

Este bloque de código es crucial en tu aplicación `app.py` y se ejecuta después de que el modelo ha predicho los pesos individuales de las partes del cangrejo (carne, vísceras, caparazón) y estas predicciones han sido transformadas de nuevo a su escala original en gramos.

```python
if sum_predicted_weights > 0:
    adjustment_factor = new_weight_input_for_prediction / sum_predicted_weights
    # Aplica el factor de ajuste
    predicted_original_scale_values = [val * adjustment_factor for val in predicted_original_scale_values]
    # Re-asegura no-negatividad después del ajuste, aunque con un factor positivo no debería hacerlos negativos
    predicted_original_scale_values = [max(0, val) for val in predicted_original_scale_values]
    st.info(f"Las predicciones se han ajustado para que su suma sea {round(new_weight_input_for_prediction, 2)}g (asumiendo que 'New Weight' es el peso total).")
else:
    st.warning("La suma de las predicciones detransformadas es cero o negativa. No se aplicó ajuste.")
    predicted_original_scale_values = [0.0] * len(ORIGINAL_TARGET_NAMES)

---
# ¿Por Qué se Usa el Factor de Ajuste en este Proyecto?

El **factor de ajuste** es un componente crítico en tu aplicación Streamlit (`app.py`) que garantiza que las predicciones del modelo sean lógicamente coherentes con la realidad física. Se aplica después de que el modelo ha predicho los pesos individuales de las partes del cangrejo (carne, vísceras, caparazón) y estas predicciones han sido transformadas de nuevo a la escala original en gramos.

## Razón Principal: Coherencia Lógica y Física

La razón fundamental para usar el factor de ajuste es asegurar que **la suma de los pesos predichos de las partes del cangrejo sea exactamente igual al peso total del cangrejo que el usuario ingresó**.

### ¿Por qué no suman exactamente sin ajuste?

Tus modelos de regresión lineal (uno para la carne, otro para las vísceras, y otro para el caparazón) son entrenados y realizan predicciones de forma **independiente** la una de la otra. Esto significa:

1.  **Modelos Separados:** El modelo que predice el `Shucked Weight` (peso de la carne) no tiene conocimiento de lo que el modelo de `Viscera Weight` (peso de las vísceras) o `Shell Weight` (peso del caparazón) está prediciendo para el mismo cangrejo.
2.  **Variabilidad Inherente:** Cada modelo intenta minimizar su propio error de predicción. Debido a la variabilidad en los datos de entrenamiento y la naturaleza probabilística de los modelos predictivos, es extremadamente improbable que, por pura coincidencia, la suma de las tres predicciones independientes sea idéntica al peso total exacto del cangrejo. Podría ser un poco más, o un poco menos.

### La Importancia del Ajuste

Considera este escenario: si un usuario ingresa "50 gramos" como el peso total de un cangrejo, y las predicciones individuales sumaran "48 gramos" o "52 gramos", esto generaría confusión e inconsistencia. Un cangrejo de 50 gramos *debe* tener partes que sumen 50 gramos.

El factor de ajuste ($\frac{\text{peso\_total\_ingresado}}{\text{suma\_de\_las\_predicciones\_individuales}}$) resuelve esto **escalando proporcionalmente** cada una de las predicciones de las partes. Si la suma original es menor que el total, el factor las aumenta; si es mayor, las reduce. Esto garantiza que la suma final de las partes predichas siempre coincida con el peso total del cangrejo ingresado.

## Impacto Crucial en el Proyecto:

1.  **Credibilidad y Confianza del Usuario:** Al presentar resultados que son físicamente imposibles (partes que no suman el total), la aplicación perdería credibilidad. El ajuste asegura que los usuarios confíen en las predicciones.
2.  **Utilidad Práctica Aumentada:** Para el "Resumen de la Producción" o cualquier análisis que dependa de la suma total, este ajuste es vital. Las cantidades de carne, material de aprovechamiento y desperdicio serán directamente utilizables y coherentes con el peso total del cangrejo procesado.
3.  **Experiencia de Usuario (UX) Superior:** La aplicación se siente más "inteligente" y precisa porque los números cuadran, lo que reduce la fricción y la confusión para el usuario.
4.  **Resultados Procesables:** Garantiza que las estimaciones de las partes del cangrejo sean inmediatamente interpretables y aplicables en un contexto real de gestión o producción.

---

En síntesis, el factor de ajuste es una medida de ingeniería de post-procesamiento que transforma predicciones estadísticamente válidas pero aritméticamente independientes en un conjunto de resultados **físicamente coherentes y altamente confiables** para el usuario final.